# Setting

In [10]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

In [ ]:
# regression
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        #step_split일 경우만 past_step과 future_step이 존재
        'data_clean_option' : "false"
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'data_clean_option' : "true",
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    }
}

# forecasting
param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testy',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        'past_step':4, #step_split일 경우만 past_step과 future_step이 존재
        'future_step':2
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    }
}

# classification
param3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testy',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "true",
        'split_mode' : 'window_split' # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
    },
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True","num_classes":6}
    }
}

In [4]:
from KETIAppDataServer2.model import dbModel
dbModel.DB_init()
from KETIAppDataServer2.API.machine_learning.model_test import set_test_model_data, test

cpu is available.
cpu is available.


In [7]:
model_name = params['model_name']
model_meta = mongodb_client.get_document_by_json('model', 'meta', {'model_name':model_name})[0]

test_X_array, test_y_array, data_y_flag_dict = set_test_model_data(params, model_meta)
result = test(model_meta, test_X_array, test_y_array, data_y_flag_dict)

window_size: 144 nan_limit_num: 72
(6048, 24) (42, 144, 24)
(42, 1) (42, 1)
features shape: torch.Size([42, 144, 24]) targets shape:  torch.Size([42, 1])

Start testing data

scale


In [8]:
result

{'df_result':         value  prediction
 0   17.370001   11.216657
 1   20.650000   10.939581
 2   11.420000   10.610282
 3   10.680000   11.173996
 4   12.439999   11.191633
 5   11.170000   10.789253
 6   24.120001   11.282571
 7   10.990000   11.263301
 8   13.760000   10.808142
 9   14.560000   10.884521
 10  14.970000   10.995320
 11  13.429999   11.031756
 12  11.570000   11.099377
 13   9.330000   11.694963
 14  15.580000   11.161883
 15  20.929998   10.955645
 16  11.990000   10.927114
 17  15.370000   11.220899
 18  10.470000   11.037809
 19  14.410000   11.135871
 20  10.160000   10.969357
 21  15.120000   10.898239
 22  12.320000   11.091141
 23  10.460000   10.624331
 24  15.060000   10.454977
 25   9.680000   10.682210
 26  10.690000   11.198977
 27  17.799997   11.188442
 28  10.690000   11.266307
 29  17.059998   11.041487
 30  20.879999   11.135978
 31  10.890000   11.033864,
 'result_metrics': {'mae': 3.3382607,
  'rmse': 4.72802066221418,
  'r2': -0.599107603790735,
 